In [266]:
import numpy as np
import cv2
from sklearn import datasets
import sklearn.datasets
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import mean_absolute_error

In [267]:
def detekcija_ivice_platoa(img):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY);
    ret, frame_thresh = cv2.threshold(gray_img, 108, 255, cv2.THRESH_BINARY)
    
#     plt.figure()
#     plt.imshow(frame_thresh, 'gray')
    
    dialated_iter1 = dilate(frame_thresh)
    dialated_iter2 = dilate(dialated_iter1)
    eroded_iter1 = erode(dialated_iter2)
    
#     plt.imshow(eroded_iter1, 'gray')
#     plt.show()

    edges_img = cv2.Canny(eroded_iter1, 50, 100, apertureSize=3)
    # minimalna duzina linije
    min_line_length = 300
    # Hough transformacija
    lines = cv2.HoughLinesP(image=edges_img, rho=1, theta=np.pi/180, threshold=50, lines=np.array([]),
                            minLineLength=min_line_length, maxLineGap=17)
    # Detektovane linije [[x1 y1 x2 y2]]:
    for i in range(len(lines)):
        cv2.line(edges_img, (lines[i][0][0], lines[i][0][1]), (lines[i][0][2], lines[i][0][3]), (255,0,0), 5);

#     plt.figure()
#     plt.imshow(edges_img, 'gray')
#     plt.show()
    
    x1 = lines[0][0][0] - 10
    y1 = lines[0][0][1]
    x2 = lines[0][0][2]
    y2 = lines[0][0][3]
    return (x1, y1, x2, y2)

In [268]:
def dilate(image):
    kernel = np.ones((3, 3)) # strukturni element 3x3 blok
    return cv2.dilate(image, kernel, iterations=1)

def erode(image):
    kernel = np.ones((3, 3)) # strukturni element 3x3 blok
    return cv2.erode(image, kernel, iterations=1)

In [269]:
def detect_cross(x, y, k, n):
    # detekcija da li je broj presao preko linije
    yy = k*x + n
    return -1 <= (yy - y) <= 1

def return_mae(given, final):
    mae = mean_absolute_error(given, final)
    return mae

In [270]:
def prebroj_ljude(index, line):
    
    path = "data/videos/video" + str(index) + ".mp4";
    capture = cv2.VideoCapture(path)
    init_frame = None
    
    brojac = 0
    while capture.isOpened(): 
        ret, new_frame = capture.read()
        if not ret:
            break
        
        if init_frame is None:
            init_frame = new_frame.copy()
            continue
            
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
        diff = cv2.absdiff(init_frame, new_frame)
        gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray, (5, 5), 0)   
        frame = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 41,10)
        dilated = cv2.dilate(frame, kernel, iterations=2)
        
        #cv2.imshow('gray', frame)
        contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
        k = (float(line[3]) - float(line[1])) / (float(line[2]) - float(line[0]))
        n = k * (float(-line[0])) + float(line[1])
        for contour in contours:
            (x, y, w, h) = cv2.boundingRect(contour)
        
            centar_rectangle_x = x + int(w / 2)
            centar_rectangle_y = y + int(w / 2)

            if detect_cross(x, y, k, n):
                    brojac = brojac + 1

        #cv2.imshow('dialated', dilated)
    return brojac

In [271]:
#PRONALAZENJE LINIJE PLATOA
path_za_detekciju_ivice = "data/videos/video1.mp4";
capture = cv2.VideoCapture(path_za_detekciju_ivice)
ret, frame = capture.read()
x1, y1, x2, y2 = detekcija_ivice_platoa(frame)
ivica_platforme = [x1, y1, x2, y2]

#PROLAZ KROZ SVAKI FAJL
rezultati = []
for i in range (1,11):
    rezultati.append(prebroj_ljude(i, ivica_platforme))
print(rezultati)

given_results = [4, 24, 17, 23, 17, 27, 29, 22, 10, 23]
print(str(return_mae(given_results, rezultati)))

[4, 14, 12, 26, 9, 18, 44, 38, 4, 29]
7.8
